In [7]:
%load_ext autoreload
%autoreload 2

from mint.model.transformer import Transformer, TransformerConfig
from mint.common import create_config, to_dict
from mint.trainer import Trainer, TrainerConfig
import os
from mint.translator import Translator
from mint.dataset import Dataset
from mint.tokenizer import Tokenizer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
DATASET_PATH = "../datasets/en_sk/"

source_tokenizer = Tokenizer.load("../tokenizers/en/")
target_tokenizer = Tokenizer.load("../tokenizers/sk/")

dataset = Dataset.load(DATASET_PATH)


In [9]:
config = create_config(TransformerConfig())
config.glob.d_model = 256
config.glob.n_heads = 8
config.glob.max_seq_len = 128 + 1
config.glob.d_feedforward = 1024
config.glob.p_dropout = 0.1

config.encoder_config.n_blocks = 6
config.encoder_config.vocab_size = 10000 + 1
config.decoder_config.n_blocks = 6
config.decoder_config.vocab_size = 10000 + 1

model = Transformer(config)
#model = Transformer.load("../pretrained/en_sk_small/")


print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")

{'n_blocks': 6, 'vocab_size': 10001, 'transformer_block_config': {'d_model': 256, 'd_feedforward': 1024, 'p_dropout': 0.1, 'attention_config': {'n_heads': 16, 'd_model': 256, 'max_seq_len': 129, 'context_window': None}}, 'embedding_config': {'vocab_size': 10001, 'd_model': 256, 'max_seq_len': 129, 'learnable_positional_embeddings': True}}
Number of parameters: 29905664


In [ ]:
trainer_config = create_config(TrainerConfig())
trainer_config.logger_config.experiment_name = "exp2"
trainer_config.warmup_steps = 1000
trainer_config.learning_rate = 5e-5
trainer_config.use_cuda = True
trainer_config.max_steps_per_epoch = 1000000 // 32 # too big dataset to run locally
trainer_config.max_steps_per_validation = 10000 // 32
trainer_config.batch_size = 32
trainer = Trainer(model, dataset, **to_dict(trainer_config), source_tokenizer=source_tokenizer, target_tokenizer=target_tokenizer)
trainer.train(10)

Train 1/10 avg loss: 9.1027:   0%|          | 13/31250 [00:03<2:04:44,  4.17it/s]

In [5]:
translator = Translator(model, source_tokenizer, target_tokenizer)

translator.translate("Good evening", max_length=128)

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
